# Coursera Capstone Project Part-1

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Convert to JSON

In [3]:
# res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# soup = BeautifulSoup(res.content,'lxml')
# table = soup.find_all('table')[0] 
# df = pd.read_html(str(table))
# print(df[0].to_json(orient='records'))

### Convert to Data Frame

In [4]:
# !pip install tabulate

In [2]:
from tabulate import tabulate
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
# tabulate(df[0], headers='keys', tablefmt='psql')
df[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
df[0].shape

(180, 3)

In [4]:
df[0].replace('Not assigned',np.nan,inplace= True)

In [5]:
df[0].dropna(inplace=True)
print(df[0].shape)
df[0].head()

(103, 3)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df[0].sort_values(by = 'Postal Code', ascending = True,inplace=True)

In [7]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
df2 = pd.read_csv('Geospatial_Coordinates.csv')

In [8]:
new_df = pd.merge(df[0],df2,how='inner',on='Postal Code')
new_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


# !conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

In [11]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [12]:
# create map of canada using latitude and longitude values
map_can = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_can)  
    
map_can